Import Dependancies

In [44]:
# Dependencies
from config import (gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

#import cleaned county data
state_data = pd.read_csv("../Matthew/Data_csv/state_life_avg.csv")
state_data["Life Exp Avg"]=((state_data["Difference in male life expectancy, 1985-2010 (years)"]+state_data["Difference in female life expectancy, 1985-2010 (years)"])/2)
state_data

,Unnamed: 0,State,"Male life expectancy, 1985 (years)","Female life expectancy, 1985 (years)","Male life expectancy, 1990 (years)","Female life expectancy, 1990 (years)","Male life expectancy, 1995 (years)","Female life expectancy, 1995 (years)","Male life expectancy, 2000 (years)","Female life expectancy, 2000 (years)","Male life expectancy, 2005 (years)","Female life expectancy, 2005 (years)","Male life expectancy, 2010 (years)","Female life expectancy, 2010 (years)","Difference in male life expectancy, 1985-2010 (years)","Difference in female life expectancy, 1985-2010 (years)",Life Exp Avg
0,0,Alabama,69.1,76.9,69.4,77.1,69.9,77.2,71.0,77.2,71.5,77.3,72.4,77.8,3.3,0.9,2.10
1,1,Alaska,70.9,77.7,72.0,78.8,72.9,78.7,74.2,78.9,75.1,79.7,75.8,80.1,4.9,2.3,3.60
2,2,Arizona,71.7,78.8,72.3,79.3,73.1,79.6,74.5,79.9,75.2,80.5,76.8,81.7,5.1,2.9,4.00
3,3,Arkansas,69.8,77.5,70.2,77.6,70.6,77.8,71.8,77.8,72.2,78.0,73.0,78.5,3.2,1.0,2.10
4,4,California,71.7,78.3,72.3,78.8,73.5,79.6,75.7,80.3,76.5,81.2,78.2,82.5,6.5,4.2,5.35
5,5,Colorado,72.9,79.0,73.6,79.6,74.4,79.9,76.0,80.2,76.5,80.9,77.6,81.7,4.7,2.7,3.70
6,6,Connecticut,72.4,78.7,73.4,79.6,74.1,79.9,75.8,80.6,76.9,81.6,78.2,82.5,5.8,3.8,4.80
7,7,Delaware,70.3,76.8,71.4,77.4,72.3,78.4,73.9,78.7,74.5,79.4,75.6,80.4,5.2,3.6,4.40
8,8,Florida,71.2,78.5,71.7,79.1,72.7,79.6,74.4,80.0,75.1,80.6,76.3,81.6,5.2,3.1,4.15
9,9,Georgia,68.7,76.4,69.3,76.9,70.4,77.3,72.1,77.6,73.0,78.2,74.4,79.2,5.7,2.8,4.25


In [21]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

# Gather Geo and County Data

In [27]:
#test space for json

# Build the endpoint URL
test_url = f"https://maps.googleapis.com/maps/api/geocode/json?address=texas&key={gkey}"

test_data = requests.get(test_url).json()
test_data['results'][0]["geometry"]["location"]["lat"]

31.9685988

In [29]:
# Run a request to endpoint and convert result to json
#geo_data = requests.get(target_url).json()

state = []
lat = []
lon = []

# Print the json
#grab county from the json
for state in state_data['State']:
    # create query url and get json
    query_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={state}&key={gkey}"
    geo_data = requests.get(query_url).json()
    #verify = geo_data['results'][0]['address_components'][3]['short_name']
    
    #if the city has data then populate lists
    if (geo_data['status'] == 'OK'):
    
        try:
            lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
            lon.append(geo_data["results"][0]["geometry"]["location"]["lng"])
            
        except IndexError:
            print(state)
            pass
    else:
        next

In [30]:
state_geo_data = pd.DataFrame({
    "State": state_data['State'],
    "Latitude": lat,
    "Longitude": lon
})

In [33]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

## Average Growth in Life Expectancy

In [46]:
# Store 'Lat' and 'Lng' into  locations 
locations = state_geo_data[["Latitude", "Longitude"]]
#life_ex = clean_weather_data['Humidity (%)']

fig1 = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=state_data["Life Exp Avg"],
                                 dissipating=False, max_intensity=8,
                                 point_radius = 3)

fig1.add_layer(heat_layer)

fig1

Figure(layout=FigureLayout(height='420px'))

## Growth in Life Expectancy by State (Females)

In [49]:

locations = state_geo_data[["Latitude", "Longitude"]]


fig2 = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=state_data["Difference in male life expectancy, 1985-2010 (years)"],
                                 dissipating=False, max_intensity=8,
                                 point_radius = 3)

fig2.add_layer(heat_layer)

fig2

Figure(layout=FigureLayout(height='420px'))